In [1]:
from util import load_user_data, task_ID_list_to_check
import pandas as pd
import numpy as np

valid_users, tp_data = load_user_data(folder_name="../anonymized_data", reserved_users=None)
user2condition = tp_data['user2condition']
condition_count = {}
for user in valid_users:
    tp_condition = user2condition[user]
    if tp_condition not in condition_count:
        condition_count[tp_condition] = 0
    condition_count[tp_condition] += 1
print(condition_count)

248 complete the NASA-TLX
248 complete the whole study
{'AP-AE': 63, 'AP-UE': 64, 'UP-UE': 60, 'UP-AE': 61}


In [2]:
user_task_confidence_dict = tp_data['confidence']
user_task_order = tp_data['task_order']
all_conditions = ["AP-AE", "AP-UE", "UP-AE", "UP-UE"]
variable_dict = {
    "condition": [],
    "planning": [],
    "execution": [],
    "confidence_planning": [],
    "confidence_execution": []
}
condition_count = {}
for user in valid_users:
    tp_condition = user2condition[user]
    tp_task_order = user_task_order[user]
    tp_list_1 = []
    tp_list_2 = []
    for index, task_id in enumerate(tp_task_order):
        confidence_planning = user_task_confidence_dict[user][task_id]["planning"]
        confidence_execution = user_task_confidence_dict[user][task_id]["execution"]
        tp_list_1.append(confidence_planning)
        tp_list_2.append(confidence_execution)
    variable_dict["condition"].append(tp_condition)
    planning, execution = tp_condition.split("-")
    variable_dict["planning"].append(planning)
    variable_dict["execution"].append(execution)
    variable_dict["confidence_planning"].append(np.mean(tp_list_1))
    variable_dict["confidence_execution"].append(np.mean(tp_list_2))

In [3]:
df = pd.DataFrame(variable_dict)
df.head()

,condition,planning,execution,confidence_planning,confidence_execution
0,AP-AE,AP,AE,2.666667,2.666667
1,AP-AE,AP,AE,4.500000,3.666667
2,AP-AE,AP,AE,5.000000,4.833333
3,AP-AE,AP,AE,4.500000,4.833333
4,AP-UE,AP,UE,3.666667,3.833333


In [9]:
from pingouin import ancova, anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
dimension = "confidence_planning"
aov = anova(dv=dimension, between=['planning', 'execution'], data=df, effsize='n2')
# aov = ancova(dv=dimension, covar=["Propensity to Trust", "Familiarity", "llm_expertise", "assistant_expertise"], between=['planning', 'execution'], data=df, effsize='n2')
print(aov.round(3))
if aov.to_dict()['p-unc'][0] <= 0.05 / 4:
    tukey = pairwise_tukeyhsd(endog=df[dimension], groups=df['condition'], alpha=0.0125)
    print(tukey)
# for condition in all_conditions:
#     print(condition, np.mean(condition_dict[condition][dimension]))
mean_value = df.groupby('condition').mean()[dimension]
print(mean_value)
print("-" * 17)

                 Source       SS     DF     MS       F  p-unc     n2
0              planning    6.076    1.0  6.076  13.962  0.000  0.053
1             execution    0.845    1.0  0.845   1.942  0.165  0.007
2  planning * execution    1.146    1.0  1.146   2.634  0.106  0.010
3              Residual  106.192  244.0  0.435     NaN    NaN    NaN
Multiple Comparison of Means - Tukey HSD, FWER=0.01 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
 AP-AE  AP-UE  -0.2495 0.1462 -0.6094  0.1104  False
 AP-AE  UP-AE  -0.4491 0.0011 -0.8134 -0.0848   True
 AP-AE  UP-UE  -0.4266 0.0023 -0.7924 -0.0607   True
 AP-UE  UP-AE  -0.1996 0.3306 -0.5625  0.1633  False
 AP-UE  UP-UE  -0.1771 0.4428 -0.5415  0.1874  False
 UP-AE  UP-UE   0.0225 0.9976 -0.3462  0.3913  False
----------------------------------------------------
condition
AP-AE    4.134921
AP-UE    3.885417
UP-AE    3.685792
UP-UE    3.708333
Name: confidence_planning, dtype: float64
-

In [8]:
from pingouin import ancova, anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
dimension = "confidence_execution"
aov = anova(dv=dimension, between=['planning', 'execution'], data=df, effsize='n2')
# aov = ancova(dv=dimension, covar=["Propensity to Trust", "Familiarity", "llm_expertise", "assistant_expertise"], between=['planning', 'execution'], data=df, effsize='n2')
print(aov.round(3))
if aov.to_dict()['p-unc'][0] <= 0.05 / 4:
    tukey = pairwise_tukeyhsd(endog=df[dimension], groups=df['condition'], alpha=0.0125)
    print(tukey)
# for condition in all_conditions:
#     print(condition, np.mean(condition_dict[condition][dimension]))
mean_value = df.groupby('condition').mean()[dimension]
print(mean_value)
print("-" * 17)

                 Source      SS     DF     MS      F  p-unc     n2
0              planning   2.735    1.0  2.735  6.872  0.009  0.026
1             execution   3.247    1.0  3.247  8.158  0.005  0.031
2  planning * execution   1.156    1.0  1.156  2.905  0.090  0.011
3              Residual  97.106  244.0  0.398    NaN    NaN    NaN
Multiple Comparison of Means - Tukey HSD, FWER=0.01 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
 AP-AE  AP-UE  -0.3621 0.0075 -0.7063  -0.018   True
 AP-AE  UP-AE  -0.3466 0.0131  -0.695  0.0017  False
 AP-AE  UP-UE  -0.4356 0.0009 -0.7854 -0.0857   True
 AP-UE  UP-AE   0.0155 0.9991 -0.3315  0.3625  False
 AP-UE  UP-UE  -0.0734 0.9162 -0.4219  0.2751  False
 UP-AE  UP-UE  -0.0889 0.8656 -0.4416  0.2637  False
----------------------------------------------------
condition
AP-AE    4.343915
AP-UE    3.981771
UP-AE    3.997268
UP-UE    3.908333
Name: confidence_execution, dtype: float64
----------